In [2]:
# downloading training annotations and metadata
!wget https://storage.googleapis.com/openimages/v6/oidv6-train-annotations-bbox.csv


--2024-09-11 11:12:12--  https://storage.googleapis.com/openimages/v6/oidv6-train-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.12.207, 172.253.118.207, 74.125.200.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.12.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2258447590 (2.1G) [text/csv]
Saving to: ‘oidv6-train-annotations-bbox.csv’

oidv6-train-annotat 100%[===================>]   2.10G  19.5MB/s    in 1m 47s  

2024-09-11 11:14:00 (20.1 MB/s) - ‘oidv6-train-annotations-bbox.csv’ saved [2258447590/2258447590]



In [3]:
# downloading validation annotations and metadata
!wget https://storage.googleapis.com/openimages/v5/validation-annotations-bbox.csv

--2024-09-11 11:14:00--  https://storage.googleapis.com/openimages/v5/validation-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.207, 74.125.200.207, 74.125.130.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25105048 (24M) [text/csv]
Saving to: ‘validation-annotations-bbox.csv’

validation-annotati 100%[===================>]  23.94M  11.3MB/s    in 2.1s    

2024-09-11 11:14:02 (11.3 MB/s) - ‘validation-annotations-bbox.csv’ saved [25105048/25105048]



In [4]:
# downloading testing annotations and metadata
!wget https://storage.googleapis.com/openimages/v5/test-annotations-bbox.csv

--2024-09-11 11:14:02--  https://storage.googleapis.com/openimages/v5/test-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.207, 74.125.200.207, 74.125.130.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 77484237 (74M) [text/csv]
Saving to: ‘test-annotations-bbox.csv’

test-annotations-bb 100%[===================>]  73.89M  22.0MB/s    in 4.5s    

2024-09-11 11:14:07 (16.6 MB/s) - ‘test-annotations-bbox.csv’ saved [77484237/77484237]



In [5]:
#downloading the dataset script
!wget https://raw.githubusercontent.com/openimages/dataset/master/downloader.py

--2024-09-11 11:14:07--  https://raw.githubusercontent.com/openimages/dataset/master/downloader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4244 (4.1K) [text/plain]
Saving to: ‘downloader.py’

downloader.py       100%[===================>]   4.14K  --.-KB/s    in 0s      

2024-09-11 11:14:08 (63.1 MB/s) - ‘downloader.py’ saved [4244/4244]



In [6]:
# required by the downloader.py file
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 9.0 MB/s eta 0:00:00


In [7]:
# intalling some dependencies
import os
import shutil

%pip install pyyaml
import yaml

In [8]:
# This fuction is used to create a custom YAML file which is used later to configure training data
def create_yolo_yaml_config(yaml_filepath, dataset_path, dataset_labels):

    data = {'path':dataset_path,
            'train': os.path.join('images', 'train'),
            'val': os.path.join('images', 'validation'),
            'names':{i:label for i, label in enumerate(dataset_labels)}
            }

    with open(yaml_filepath, 'w') as fp:
    # set sort_keys = False to preserve the order of keys
        yaml.dump(data, fp, sort_keys=False)


# This function returns a dict with the names and IDs of selected object we want to train our model to detect
def get_class_id(classes_file, class_names):
    id_name_dict = {}
    with open(classes_file, 'r') as f:
        for line in f:
            id, label = line.split(',')
            label = label.strip()
            #print(label)
            if label in class_names:
                print(label)
                id_name_dict[label] = id

    return id_name_dict

In [9]:
#here we are defining the classes we require from the dataset and creating the config.yaml file for its configuration
names = ['Ant', 'Insect']

dataset_path = os.path.abspath(os.path.join('.', 'data'))

yaml_filepath = os.path.join('.', 'config.yaml')

create_yolo_yaml_config(yaml_filepath, dataset_path, names)

In [11]:
# geting the class IDs of the target objects, all detectable objects and their IDs are pre-determined by OpenImage in class-descriptions-boxable.csv file
class_ids = get_class_id('./class-descriptions-boxable.csv'
                         ,names)
print(class_ids)
print(names)

Insect
Ant
{'Insect': '/m/03vt0', 'Ant': '/m/0_k2'}
['Ant', 'Insect']


In [12]:
# Creating a list of annotated images to be downloaded by the downloader script download.py


train_bboxes_filename = os.path.join('.', 'oidv6-train-annotations-bbox.csv')
validation_bboxes_filename = os.path.join('.', 'validation-annotations-bbox.csv')
test_bboxes_filename = os.path.join('.', 'test-annotations-bbox.csv')

image_list_file_path = os.path.join('.', 'image_list_file.txt')

image_list_file_list = []
for j, filename in enumerate([train_bboxes_filename, validation_bboxes_filename, test_bboxes_filename]):
    print(filename)
    with open(filename, 'r') as f:
        line = f.readline()
        while len(line) != 0:
            id, _, class_name, _, x1, x2, y1, y2, _, _, _, _, _ = line.split(',')[:13]
            if class_name in list(class_ids.values()) and id not in image_list_file_list:
                image_list_file_list.append(id)
                with open(image_list_file_path, 'a') as fw:
                    fw.write('{}/{}\n'.format(['train', 'validation', 'test'][j], id))
            line = f.readline()

        f.close()

./oidv6-train-annotations-bbox.csv
./validation-annotations-bbox.csv
./test-annotations-bbox.csv


In [13]:
#creating directories to save dataset
DATA_ALL_DIR = os.path.join('.', 'data_all') # directory that contains all downloaded data selected from the list
DATA_OUT_DIR = os.path.join('.', 'data') # directory that contains data reorganized in YOLO format
os.makedirs(DATA_ALL_DIR)
os.makedirs(DATA_OUT_DIR)

In [14]:
# run the donwloader script in order to download data related to the target objects and according to the image_list_file.txt
!python downloader.py ./image_list_file.txt --download_folder=./data_all --num_processes=5

In [15]:
# Create a dataset in the yolo format with containing train, test and validation data


for dir_ in ['images', 'labels']:
    for set_ in ['train', 'validation', 'test']:
        new_dir = os.path.join(DATA_OUT_DIR, dir_, set_)
        if os.path.exists(new_dir):
            shutil.rmtree(new_dir)
        os.makedirs(new_dir)

In [16]:
# Filling the dataset with the appropriate images and labels in the appropriate format

for j, filename in enumerate([train_bboxes_filename, validation_bboxes_filename, test_bboxes_filename]):
    set_ = ['train', 'validation', 'test'][j]
    print(filename)
    with open(filename, 'r') as f:
        line = f.readline()
        while len(line) != 0:
            id, _, class_name, _, x1, x2, y1, y2, _, _, _, _, _ = line.split(',')[:13]
            if class_name in list(class_ids.values()):

                if not os.path.exists(os.path.join(DATA_OUT_DIR, 'images', set_, '{}.jpg'.format(id))):

                    shutil.copy(os.path.join(DATA_ALL_DIR, '{}.jpg'.format(id)),
                                os.path.join(DATA_OUT_DIR, 'images', set_, '{}.jpg'.format(id)))

                with open(os.path.join(DATA_OUT_DIR, 'labels', set_, '{}.txt'.format(id)), 'a') as f_ann:
                    # class_id, xc, yx, w, h
                    #
                    x1, x2, y1, y2 = [float(j) for j in [x1, x2, y1, y2]]
                    xc = (x1 + x2) / 2
                    yc = (y1 + y2) / 2
                    w = x2 - x1
                    h = y2 - y1

                    # class id = 0 if 'Ant' and 1 if 'Insect'
                    name = [k for k, v in class_ids.items() if v == class_name][0]
                    class_id = names.index(name)

                    #*****
                    f_ann.write('{} {} {} {} {}\n'.format(class_id, xc, yc, w, h))
                    f_ann.close()

            line = f.readline()

./oidv6-train-annotations-bbox.csv
./validation-annotations-bbox.csv
./test-annotations-bbox.csv


In [17]:
#training the object detection model
%pip install ultralytics
import ultralytics
ultralytics.checks()




Ultralytics YOLOv8.2.91 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.2/112.6 GB disk)


In [19]:
from ultralytics import YOLO


# Load a model
model = YOLO("yolov8n.pt")

# Use the model for training
results = model.train(data='./config.yaml',
                      epochs=10)  # train the model

Ultralytics YOLOv8.2.91 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./config.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

train: Scanning /content/data/labels/train.cache... 6461 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6461/6461 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/data/labels/validation.cache... 210 images, 0 backgrounds, 0 corrupt: 100%|██████████| 210/210 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      10/10      3.55G      1.023       1.07       1.38         15        640: 100%|██████████| 404/404 [03:07<00:00,  2.15it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:04<00:00,  1.55it/s]

                   all        210        280      0.506      0.447      0.464      0.328



10 epochs completed in 0.537 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.2.91 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:06<00:00,  1.13it/s]


                   all        210        280      0.507      0.447      0.464      0.329
                   Ant         12         30      0.367      0.167      0.181      0.117
                Insect        198        250      0.647      0.728      0.747      0.542
Speed: 0.2ms preprocess, 4.2ms inference, 0.0ms loss, 5.2ms postprocess per image
Results saved to runs/detect/train2


In [22]:
# Create a zip archive of the 'runs' folder

import shutil


shutil.make_archive('train_results', 'zip', './runs')


'/content/train_results.zip'